### For Run in Google Colab

In [1]:
%%capture
!pip install fasttext
!pip install emoji

You should consider upgrading via the 'C:\Users\Roy\.virtualenvs\ML-Developer\Scripts\python.exe -m pip install --upgrade pip' command.


  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171047 sha256=f2b82e7aa3740f0530f510b5df135d1f552f5118db0dd1d69b03ac0a714355c6
  Stored in directory: c:\users\roy\appdata\local\pip\cache\wheels\5e\8c\80\c3646df8201ba6f5070297fe3779a4b70265d0bfd961c15302
Successfully built emoji


In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive')
os.chdir("/content/drive/MyDrive/Healthcare/")

### Import Library

In [2]:
import os
import csv
import time
import json
import emoji
import fasttext
import numpy as np
import pandas as pd
import string
# import mlflow
import json
import pickle

from datetime import date

from sklearn.model_selection import train_test_split
from sklearn.metrics import (f1_score,
                             precision_score,
                             recall_score, 
                             accuracy_score)

### Train Model

In [3]:
trainset_path = './dataset/train_fasttext.txt'
testset_path = './dataset/test_dataset.csv'

model = fasttext.train_supervised(
                    input = trainset_path,
                    autotuneModelSize = "10M",
                    autotuneDuration = 600,
                    verbose = 1
  )
  

### Evaluate the model

In [4]:
# Do inference on dataset
test_dataset = pd.read_csv(testset_path)

test_dataset["label predict"] = test_dataset['TEXT'].apply(
    lambda x:model.predict(x,k=1)[0][0].replace("__label__",""))
test_dataset["score predict"] = test_dataset['TEXT'].apply(
    lambda x:model.predict(x,k=1)[1][0])

In [5]:
test_dataset

,TEXT,LABEL,label predict,score predict
0,fatigue mood swings weight loss restlessness s...,hyperthyroidism,hyperthyroidism,0.515141
1,skin rash chills joint pain vomiting fatigue h...,dengue,dengue,0.517676
2,joint pain vomiting fatigue yellowish skin dar...,hepatitis-d,hepatitis-d,0.359309
3,chills vomiting fatigue weight loss cough high...,tuberculosis,tuberculosis,0.620065
4,skin rash high fever blister red sore around n...,impetigo,impetigo,0.769440
...,...,...,...,...
1471,continuous sneezing shivering chills watering ...,allergy,allergy,0.929228
1472,cramps bruising obesity swollen legs swollen b...,varicose-veins,varicose-veins,0.735387
1473,chills cough high fever breathlessness sweatin...,pneumonia,pneumonia,0.778275
1474,joint pain neck pain knee pain hip joint pain ...,osteoarthristis,osteoarthristis,0.873282


In [13]:
from tqdm.notebook import tqdm
from tqdm import tqdm

tqdm.pandas()

from sklearn.metrics import (f1_score,
                             precision_score,
                             recall_score, 
                             accuracy_score)

def check_accuracy(model, text, label, n):
    pred = model.predict(text,k=8)[0]
    pred = [lbl.replace("__label__","") for lbl in pred]
    if label in pred[:n]: return True
    return False

In [14]:
accuracy = accuracy_score(
                  test_dataset["LABEL"],
                  test_dataset["label predict"]
  )
precision = precision_score(
                  test_dataset["LABEL"],
                  test_dataset["label predict"],
                  average="weighted"
  )
recall = recall_score(
                  test_dataset["LABEL"],
                  test_dataset["label predict"],
                  average="weighted"
  )
f1 = f1_score(test_dataset["LABEL"],
              test_dataset["label predict"],
              average="weighted")

test_dataset["top1"] = test_dataset.progress_apply(
                    lambda x: check_accuracy(model = model,
                                              text = x["TEXT"],
                                              label = x["LABEL"],
                                              n = 1), axis=1)

test_dataset["top3"] = test_dataset.progress_apply(
                    lambda x: check_accuracy(model = model,
                                              text = x["TEXT"],
                                              label = x["LABEL"],
                                              n = 3), axis=1)

test_dataset["top8"] = test_dataset.progress_apply(
                    lambda x: check_accuracy(model = model,
                                              text = x["TEXT"],
                                              label = x["LABEL"],
                                              n = 8), axis=1)

top1 = test_dataset["top1"].value_counts(normalize=True).loc[True]
top3 = test_dataset["top3"].value_counts(normalize=True).loc[True]
top8 = test_dataset["top8"].value_counts(normalize=True).loc[True]

metric_result = {"top1":top1,
                  "top3":top3,
                  "top8":top8,
                  "accuracy":accuracy,
                  "precision":precision,
                  "recall":recall,
                  "f1":f1}
  





100%|██████████| 1476/1476 [00:00<00:00, 10250.74it/s]




100%|██████████| 1476/1476 [00:00<00:00, 11267.36it/s]




100%|██████████| 1476/1476 [00:00<00:00, 8531.50it/s]


In [15]:
metric_result

{'top1': 0.9911924119241192,
 'top3': 1.0,
 'top8': 1.0,
 'accuracy': 0.9911924119241192,
 'precision': 0.991874940714917,
 'recall': 0.9911924119241192,
 'f1': 0.9911393391031353}

In [16]:
# Check incorrect label
test_dataset[test_dataset['label predict'] != test_dataset['LABEL']]

,TEXT,LABEL,label predict,score predict,top1,top3,top8
36,itching skin rash burning micturition spotting...,drug-reaction,fungal-infection,0.424881,False,True,True
125,itching skin rash burning micturition spotting...,drug-reaction,fungal-infection,0.424881,False,True,True
147,vomiting fatigue yellowish skin dark urine nau...,hepatitis-d,chronic-cholestasis,0.311154,False,True,True
202,back pain weakness in limbs neck pain dizziness,cervical-spondylosis,dimorphic-hemmorhoidspiles,0.428165,False,True,True
256,joint pain vomiting fatigue yellowish skin dar...,hepatitis-d,hepatitis-a,0.336585,False,True,True
338,itching skin rash burning micturition spotting...,drug-reaction,fungal-infection,0.424881,False,True,True
506,vomiting fatigue yellowish skin dark urine nau...,hepatitis-d,chronic-cholestasis,0.311154,False,True,True
514,itching skin rash burning micturition spotting...,drug-reaction,fungal-infection,0.424881,False,True,True
821,back pain weakness in limbs neck pain dizziness,cervical-spondylosis,dimorphic-hemmorhoidspiles,0.428165,False,True,True
838,itching skin rash burning micturition spotting...,drug-reaction,fungal-infection,0.424881,False,True,True


In [17]:
model.save_model("model/model.ftz")

In [18]:
import json

metric_result

with open('model/metric_result_fasttext.json', 'w') as outfile:
    json.dump(metric_result, outfile)

### Try the model

In [19]:
model = fasttext.load_model("model/model.ftz")
model.predict("swelling of stomach knee pain", k=4)

(('__label__cervical-spondylosis',
  '__label__alcoholic-hepatitis',
  '__label__osteoarthristis',
  '__label__gerd'),
 array([0.41831651, 0.12049904, 0.11539995, 0.09048676]))